Custiomers and Orders - The lifeblood of any business!


In [1]:
import requests

def import_data_files():
  r = requests.get('https://raw.githubusercontent.com/anyoneai/notebooks/main/customers_and_orders/data/customers.csv')
  with open('./sample_data/customers.csv', 'wb') as f:
    f.write(r.content)

  r = requests.get('https://raw.githubusercontent.com/anyoneai/notebooks/main/customers_and_orders/data/orders.csv')
  with open('./sample_data/orders.csv', 'wb') as f:
    f.write(r.content)

import_data_files()
print("Customers and orders CSV files have been added './sample_data'")

Customers and orders CSV files have been added './sample_data'


# Processing Customers data

The sample customer data in 'customers.csv' file has just 5 columns: CustomerId, First Name, Last Name, City and State

![Data sample](https://raw.githubusercontent.com/anyoneai/notebooks/main/customers_and_orders/images/customers.png)


**Question 1:** How many customers are in the file?

In [2]:
from os.path import exists
import csv

datafile = "./sample_data/customers.csv"
if not exists(datafile):
  raise SystemExit("You should run the first code cell and download the dataset files!")

with open(datafile, 'r') as fl:
  csvreader = csv.reader(fl, delimiter=',')

  loop_count = 0

  for row in csvreader:
    loop_count += 1

  loop_count -= 1 # don't count header

  # BETTER SOLUTION:
  # sum_count = sum(1 for row in csvreader) - 1


print(loop_count)

602


**Question 2:** In how many different states do the customers live?

In [3]:
import pandas as pd

data = pd.read_csv(datafile)

# normalize data
data['State'] = data['State'].str.strip().str.upper()

unique_state_count = data.iloc[:,4].drop_duplicates().size

print(unique_state_count)


14


**Question 3** What is the state with most customers?

In [4]:
most_state = data.value_counts('State').keys()[0]

print(most_state)

CA


**Question 4** What is the state with the least customers?

In [5]:
fewest_state = data.value_counts('State').keys()[-1]

print(data.value_counts('State'))

State
CA    569
NV      8
AZ      6
CO      3
FL      3
NM      3
TX      2
UT      2
ID      1
IN      1
MA      1
NH      1
OR      1
WA      1
Name: count, dtype: int64


**Question 5:** What is the most common last name?

In [6]:
# normalize data
data['LastName'] = data['LastName'].str.strip().str.title()

common_lname = data.value_counts('LastName').keys()[0]

print(common_lname)

Smith


Processing Orders data

The second sample files contains orders placed by customers from the first file.

The file contains the following columns: CustomerID, OrderID, Date, OrderTotal, ProductName, Price

![Data sample](https://raw.githubusercontent.com/anyoneai/notebooks/main/customers_and_orders/images/orders.png)




In [7]:
# Question 1: How many unique orders are in the orders.csv file?

orders_file = "./sample_data/orders.csv"
order_data = pd.read_csv(orders_file)

unique_orders = order_data.iloc[:,1].drop_duplicates().size

print(unique_orders)

16672


In [8]:
# Question 2: What is the average number of items per order (rounded to two decimal places)?

total_items = len(order_data)

average_items = round((total_items / unique_orders), 2)

print(average_items)

1.76


In [9]:
# Question 3: What is the highest number of items per order?

most_items = order_data.groupby(['OrderID'])['ProductName'].count().sort_values().max()

print(most_items)

35


In [10]:
#Question 4: What is the number of orders placed in October 2021?

order_data['Date'] = pd.to_datetime(order_data['Date'], format='%Y-%m-%d %H:%M:%S.%f')
groups = order_data.groupby(by=[order_data.Date.dt.month, order_data.Date.dt.year])
october_items = groups.get_group((10.0, 2021.0))
october_orders = october_items['OrderID'].drop_duplicates().size

print(october_orders)

267


In [11]:
# Question 5: Which customer spent the most amount of money in 2021?

order_data.groupby([order_data.Date.dt.year, 'CustomerID'])['Price'].sum()
order_data.groupby([order_data.Date.dt.year]).get_group((2021.0)).groupby(['CustomerID'])['Price'].sum().sort_values()

,Price
CustomerID,
5370388,85.0
5370422,90.0
5415323,100.0
5370884,100.0
5415286,100.0
...,...
4300438,4960.0
5100984,5225.0
5464,5470.0


In [12]:
# Question 6: Historically, what is the best month for sales?

order_data.groupby(by=[order_data.Date.dt.month])['Price'].sum().sort_values(ascending=False).idxmax()

1.0